In [ ]:
import pandas as pd

df = pd.read_parquet("hf://datasets/ButterChicken98/plantvillage-image-text-pairs/data/train-00000-of-00001.parquet")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df.head()

,image,caption,captions
0,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,Tomato healthy,[A vibrant green and healthy tomato leaf with ...
1,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,Tomato Late blight,[A tomato leaf showing dark brown lesions and ...
2,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,Tomato healthy,[A vibrant green and healthy tomato leaf with ...
3,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,Tomato mosaic virus,[A tomato leaf with mosaic-like patterns of li...
4,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,Pepper bell healthy,"[A fresh green bell pepper leaf with a smooth,..."


In [ ]:
df = df.drop(columns=["image"])
print(df.head())
print(df.shape)

               caption                                           captions
0       Tomato healthy  [A vibrant green and healthy tomato leaf with ...
1   Tomato Late blight  [A tomato leaf showing dark brown lesions and ...
2       Tomato healthy  [A vibrant green and healthy tomato leaf with ...
3  Tomato mosaic virus  [A tomato leaf with mosaic-like patterns of li...
4  Pepper bell healthy  [A fresh green bell pepper leaf with a smooth,...
(20638, 2)


In [ ]:
df = df.explode("captions")
print(df.head())
print(df.shape)

              caption                                           captions
0      Tomato healthy  A vibrant green and healthy tomato leaf with s...
0      Tomato healthy  A healthy Solanum lycopersicum leaf, free of d...
0      Tomato healthy  A fresh tomato leaf outdoors, glowing in sunli...
0      Tomato healthy  A clean and healthy tomato leaf image, perfect...
1  Tomato Late blight  A tomato leaf showing dark brown lesions and w...
(82552, 2)


In [ ]:
caption = df["caption"].unique().tolist()
caption, len(caption)

(['Tomato healthy',
  'Tomato Late blight',
  'Tomato mosaic virus',
  'Pepper bell healthy',
  'Potato Early blight',
  'Tomato Early blight',
  'Tomato YellowLeaf Curl Virus',
  'Tomato Target Spot',
  'Pepper bell Bacterial spot',
  'Tomato Septoria leaf spot',
  'Tomato Spider mites Two spotted spider mite',
  'Tomato Bacterial spot',
  'Potato Late blight',
  'Tomato Leaf Mold',
  'Potato healthy'],
 15)

In [ ]:
df.to_csv("dataset.csv", index=False)

In [ ]:
df = pd.read_csv("dataset.csv")

In [ ]:
df.head()

,caption,captions
0,Tomato healthy,A vibrant green and healthy tomato leaf with s...
1,Tomato healthy,"A healthy Solanum lycopersicum leaf, free of d..."
2,Tomato healthy,"A fresh tomato leaf outdoors, glowing in sunli..."
3,Tomato healthy,"A clean and healthy tomato leaf image, perfect..."
4,Tomato Late blight,A tomato leaf showing dark brown lesions and w...


In [ ]:
num_labels = 15
data_col_name = "captions"
label_col_name = "caption"

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()
encoder.fit(df[label_col_name].tolist())
df["label"] = encoder.transform(df[label_col_name].tolist())

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test = train_test_split(df, train_size=0.8)

In [ ]:
from datasets import Dataset

In [ ]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [ ]:
model_name = "distilbert-base-uncased"

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_fn(data):
  return tokenizer(data["captions"], truncation=True)

In [ ]:
tokenized_train = train_dataset.map(tokenize_fn, batched=True)
tokenized_test = test_dataset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/66041 [00:00<?, ? examples/s]

Map:   0%|          | 0/16511 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels= 15)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
import evaluate
from transformers import TrainingArguments, Trainer
import numpy as np

In [ ]:
eval_metrics = evaluate.load("accuracy")

In [ ]:
def metrics(eval_pred):
  logits, labels = eval_pred
  prediction = np.argmax(logits, axis=1)
  return eval_metrics.compute(predictions=prediction, references=labels)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    learning_rate=0.00005,
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=2,
    weight_decay=0.01,
    report_to="none"

)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics= metrics
)

/tmp/ipython-input-4236874995.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
8256,0.021900
16512,0.000000


TrainOutput(global_step=16512, training_loss=0.010964015608009532, metrics={'train_runtime': 686.5522, 'train_samples_per_second': 192.384, 'train_steps_per_second': 24.051, 'total_flos': 922071682212750.0, 'train_loss': 0.010964015608009532, 'epoch': 2.0})

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict_caption(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    pred_id = torch.argmax(logits, dim=-1).item()
    label = encoder.inverse_transform([pred_id])[0]
    return label


In [ ]:
test_text = "A tomato leaf with small, dark circular spots, indicative of target spot disease."
predicted_label = predict_caption(test_text)
print("Predicted Label:", predicted_label)


Predicted Label: Tomato Target Spot


In [ ]:
import joblib

# Save model weights and tokenizer
model.save_pretrained("./plant_classifier_model")
tokenizer.save_pretrained("./plant_classifier_model")

# Save label encoder
joblib.dump(encoder, "./plant_classifier_model/label_encoder.pkl")

print("✅ Model, tokenizer, and label encoder saved successfully!")



✅ Model, tokenizer, and label encoder saved successfully!


In [ ]:
!zip -r plant_classifier_model.zip plant_classifier_model


  adding: plant_classifier_model/ (stored 0%)
  adding: plant_classifier_model/special_tokens_map.json (deflated 42%)
  adding: plant_classifier_model/vocab.txt (deflated 53%)
  adding: plant_classifier_model/model.safetensors (deflated 8%)
  adding: plant_classifier_model/tokenizer.json (deflated 71%)
  adding: plant_classifier_model/tokenizer_config.json (deflated 75%)
  adding: plant_classifier_model/label_encoder.pkl (deflated 82%)
  adding: plant_classifier_model/config.json (deflated 60%)


In [ ]:
from google.colab import files
files.download("plant_classifier_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>